<a href="https://colab.research.google.com/github/saraelshawa/CSC324/blob/master/NLP_lecture3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 3 

## conditioned language models
- not just generate text, generate text according to some specification

Input x --> Output Y (Text) ---> Task 

structured data -> nl description --> NL Generation (data-to-text generation) how you get weather reports, responses from google assistant

english --> japanese --> Translation 

document --> short description --> Summarization

utterance --> response  --> Response generation

image --> text ---> image captioning

speech --> transcript --> speech recognition 

they all follow the same format, so we can use very similar models

## Calculating probability of a sentence
- P(x) prob of next word, given the context 
## condition language models
- P(y|x) we condition on input x and all the previous tokens until j-1 to predict j, x is added context

## One Type ofLanguage Model Mikolov et al. 2011
going through multiple LSTMs (sutskever et al 2014)
for translation, we feed in whatever we want to translate and end of sentence symbol (**encoder**), then we predict all of the next words and generate outputs(**decoder**).

ex. feed in japanese sentence, outputs english sentence

# Methods of Generation
## the generation model
- we have a model of P(Y|X), how do we use it to generate a sentence 
- Two methods:
    - **Sampling**: try to generate a random sentence accfording to prob distrub, when you neeed to generate a variety of different sentences, ex. chatbot and instead of when the user says hi, it would sometimes say hi, hello, greetings. Another reason is to see what the model thinks are reasonable sentence. if uyou only pick the best sentence you wont see what the model is thinking
    - **Argmax**: try to generate the sentence with the highest probability. example in translation you will only show the user the best output and only one output
  
## Ancestral Sampling:
- Randomly generate words one-by-one
- while we havent reach end of sentence sybmol, calculate probability dist over the next word and randomly sample a word from there
- an exact method for sampling p(y|x), no further work needed

## Greedy Search (searching for the best output)
- one by one, pich the single highest probability word
- while we havent reach end of sentence symbol, pick highest scoring word given our context
- not exact, real problems:
    - will often generate the 'easy' words first
    - will prefer multiple common words to one rare word (example: will choose" "new" if you have New Jersey, New York, compared to Pittsburgh)
    - you will end up getting suboptimal solutions doing this

## Beam Search 
- instead of picking one high-probability word, maintain several paths (in the locations example, we would pick "new" and "pittsburgh", so not just "new" like in our greedy search 
- can find shorter sentences

# How do we Evaluate? 
## human evaluation
- as a human to do evaluation
- is it adequate? (meaning correct) 
- is it fluent? (is the grammar correct)
- betteR? rank-based evaluation. being adequate and not fluent is ok "the bob visited the mary", other way around no, "mary visited bob"
- inal goal, but slow,expensice and sometimes inconsisten so we use automatic evaluation

##basic evaluation paradigm automatic evaluiation
- use parallel test set 
- use system to generate translations
- compare target translations w/ reference

## Bleu (first popular machine evalution)
- works by comparing n-gram overlap w/reference
-the more thing that are included in the system output that are also in the reference will increase the score
- compare system output with reference and look for sufface-level match
- pros: easy to use good for measuring system  improvement
- cons: often doesn't match human eval, bad for comparing v different systems

## METEOR
- like bleu in overall principle, with many other tricks, like consider paraphrases, reordering, and function word/content word diff
- pros: generally significantly better than bleu, esp for high-resource languages
- cons: requires extra resources for new languages (although these can be made automatically), and more complicated 

## Embedding-based Metrics (BERTScore, BLEURT)
- use either similarity between nerual net embeddings (bertscorer) or a trained nerual network model to predict human scores (BLEURT), or use neural net to predict what a human would say 
- pros: strong performance (at least on in-domain data) 
- cons: required trained embeddings, is not interpretable and may not generalize to new domains

##perplexity (another method to test our models)
- calculat ethe perplexity of the words in theheld-out set without doing generation
- Pros: naturally solves multiple-reference problem!
- Cons: doesn't consider decoding or actually generating output. so if you had a bug you cannot detect it with perplexity
- maybe reasonable for problems with lots of ambiguity like dialogue

# Debugging Generation
## Debugging Search (1)
- your decoding code (search code) should get the same score as loss calculation
- test this:
  -calculate loss of reference
  - perform forced deocoding, where you decode, but tell your model the reference word at each tiem step
  - the score of these two should be the same 

## (2)
- as you make search better, the modeol score should get  better  (almost all time)
- search with varying beam sizes and make sure you get a better overall model score with larger sizes 

## Look at your data!
- decoding problems can often be detected by looking at outputs and realizing something is weong
- ex. the first word of the sentence is droppped every time
  - went to he store yesterday
  - bought a dog

##Quantitative analysis
- measure gains quanititaitvely. what is the phenonmenon you chose to focus on? is that phenonmenon getting better? 
  - you focused on low-ffrequency words: is accuracy on low freqyency words increasing?
  - you focused on syntax: is syntax or word ordering getting better, are you doing better on long-distance dependencies? 
  - you focused on search: are you reducing the number of search errors

toolkit: neulab/compare-mt, automatically generates html report, outputs accuracies
- read:climbing towards nlu, on meaning, form -- octopus tests
# Mismatch b/t optimized function and evaluation metric

## loss function, evaluation metric
- it is v common to optimize for maximum likelihood in training for that the accuracy can get worse,
-A stark example (koehn and knowles 2017) better score (better model score) can result in worse bleu score)
- probability is not v correlated with how well the output is

## Managing loss functions/eval metric differences
- most prrincipled way: use structured prediciotn techniques (in his class)

##A simple method: early stopping w/ eval metric
- rememberthis graph: difference between number of iterations for best loss vs. best eval 

# Attention 
## Sentence representations:
- problem: you can't cram the meaning of a whole sentence in a single vector by ray mooney
- but what if we could use multiple vectors, based on the length of the sentence
- if sentence of length 4, cram it into 4 vectors
- more realistic than saying one vector is for a sentence of length 2 and length 40

##Basic Idea:
- encode each word in the sentence into a vector
-when decoding, perfom a linear combination of these vectors, weighted by attention weights - higher attention weight means u focus on this vector more
- use this combination in picking thenext word

-use 'query' vector (decoder state) and 'key' vectors (all encoder states
- for each query-key pair, calculate weight
- normalize to add to one using softmax function
- results like prob distribution over the words in the input sentence. tells us the first one is v important
- then we combine together value vectors (usually encoder states, like key vectors) by taking the weighted sum, and combine them into a single vector
- we can use this in any part of the model you like (text classifciation, generation, etc)

## Atetention score functions (1) 
- q is query, k is key
- multi-layer perceptron --> feed to neural net with a weight matrix nonlinearlity then another wieight metric
  - flexbile, oftern very  good with large data
- bilinear 
  -take the query and multiply by wieght matrix then by key vector
- dot product 
  - no prameters! reuqires sizes to be the same
- scaled dot product
  -prooblem: scale of dot product increases as dimensions get larger
  -fix: scale by size of the vector 

- let's try it out attention.py

# improvements to attention and transformers
## intra-attention/ self attention
- each element in the setnece attends to other elements -< context senestivie encodings
- row and columns are identical, you usually focus on yourself but you can pull in information from other words

##Multi-header attention
- idea: multiple attention "heads" focus on different parts  of the sentence
- eg. different heads for "copy" vs regular
- or multiple independently learned heads (what the transformer paper did)
- or one head for every hiden node (too comp. expensive)

## Summary of the "transformer"
-a sequence to sequence model based entirely on attention
- strong results on standard WMT datasets
- fast: only matrix multiplication

## Attention tricks
- **self attention**: each layer combines words with others
- **multi headed attention** : 8 attention heads learned indepen.
- **normalziated dot-product attention**: remove vbias in dot produce when using large entworks
- **positional encodings**: makes sure that even if we dont have RNN still can distringus psition allows you to distinguish between "the" in the first psotion or in the firfth possiton

## Trianing tricks
-**layered normaliztion**: help ensure that layers remain in reasonabl range
- **specialized training schedule**: adjust default learning rate of the adam optimizer (dec learning late afteR) 
- **label smoothing**:  insert some uncertainty in training process, by smoothing out prob of all the things you can predict
- **masking for efficient training** 















In [ ]:
(())